In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re

def get_position(cabin):
    position_search = re.search(r'(\w)/\d+/\w',cabin)
    if position_search:
        return position_search.group(1)
    return ""


def get_position2(cabin):
    position_search = re.search(r'\w/\d+/(\w)',cabin)
    if position_search:
        return position_search.group(1)
    return""

def add_label(df):
    df["Age"].describe()
    df["Age"].fillna(df["Age"].median(),inplace=True)
    col = ["Age"]
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(df[col])
    df[col] = scaler.transform(df[col])

    def get_id(PassengerId):
        id_search = re.search(r'(\d{4})\w\d{2}',PassengerId)
        
        if id_search:
            return id_search.group(1)
        return ""

    df["id_1"]=df["PassengerId"].apply(get_id)
    col = ["id_1"]
    scaler.fit(df[col])
    df[col] = scaler.transform(df[col])

    df["Group_num"] = df.groupby('id_1')['PassengerId'].transform('count')
    from sklearn.preprocessing import PowerTransformer
    ptb = PowerTransformer(method='box-cox')
    pty = PowerTransformer(method='yeo-johnson')
    col = ["Group_num"]
    ptb.fit(df[col])
    df[col] = ptb.transform(df[col])

    def get_id(PassengerId):
        id_search = re.search(r'\d{4}\w(\d{2})',PassengerId)
        
        if id_search:
            return id_search.group(1)
        return ""

    df["id_2"]=df["PassengerId"].apply(get_id)
    from sklearn.preprocessing import PowerTransformer
    ptb = PowerTransformer(method='box-cox')
    pty = PowerTransformer(method='yeo-johnson')
    col = ["id_2"]
    ptb.fit(df[col])
    df[col] = ptb.transform(df[col])

    return df

In [ ]:
import time 
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import sklearn.metrics 
import pandas as pd
import optuna
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, PowerTransformer
from sklearn.impute import SimpleImputer

train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
train = add_label(train)

list_cols = ["HomePlanet","CryoSleep","Destination","VIP"]
list_cat = ["Area", "Side", 'isChild', 'isOld']
target = "Transported"

train["Cabin"].fillna('Z/00/Z',inplace=True)
train["Area"] = train["Cabin"].apply(get_position)
train["Side"] = train["Cabin"].apply(get_position2)
train['GroupId'] = train['PassengerId'].str[:4]
train['PeopleInGroup1'] = train.groupby('GroupId')['PassengerId'].transform('count')
train['PersonalId'] = train['PassengerId'].str[-2:]
train['PeopleInGroup2'] = train.groupby('PersonalId')['PassengerId'].transform('count')
train['isChild'] = train['Age'].apply(lambda x: 1 if x < 10 else 0)
train['isOld'] = train['Age'].apply(lambda x: 1 if x > 60 else 0)

del train["Cabin"], train["Name"]

oe = OrdinalEncoder()
oe.fit(train[list_cat])
train.loc[:,list_cat]  = oe.transform(train[list_cat])

ohe = OneHotEncoder(sparse=False)
ohe.fit(train[list_cols])

ptr = PowerTransformer()
num_col = list(set(train.columns)-set(list_cols)-set(list_cat)-set([target]))
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean', fill_value=None)
train[num_col] = imp_mean.fit_transform(train[num_col])
ptr.fit(train[num_col])
train.loc[:,num_col]  = ptr.transform(train[num_col])

df_new = pd.DataFrame(ohe.transform(train[list_cols]), 
                      columns=ohe.get_feature_names(), 
                      dtype=np.int64)
train = train.drop(list_cols, axis=1)

df_session_ce_onehot = pd.concat([train, df_new] ,axis=1)
print(df_session_ce_onehot.head(3))
y = df_session_ce_onehot[target].values
X = df_session_ce_onehot[df_session_ce_onehot.columns[df_session_ce_onehot.columns != target]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

K_fold = KFold(n_splits=5, shuffle=True,  random_state=42)

row_no_list = list(range(len(y_train)))

def objective(trial):
    param = {
        'objective': 'binary',
        'metric': trial.suggest_categorical('metric', ['binary_error',"binary_logloss"]),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 100),#7
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth':trial.suggest_int('max_depth',1,100),
        "verbose": -1,
    }

    accuracy = 0
    for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train):
        X_train_cv = X_train[train_cv_no, :]
        y_train_cv = pd.Series(y_train)[train_cv_no]
        X_test_cv = X_train[eval_cv_no, :]
        y_test_cv = pd.Series(y_train)[eval_cv_no]

        train_data = lgb.Dataset(X_train_cv, label=y_train_cv)
        eval_data = lgb.Dataset(X_test_cv, label=y_test_cv, reference= train_data)

    
        gbm = lgb.train(
                param,
                train_data,
                num_boost_round=1000,
                )
        preds = gbm.predict(X_test_cv)
        pred_labels = np.rint(preds)
        ac_cv = sklearn.metrics.accuracy_score(y_test_cv, pred_labels)
        if accuracy < ac_cv: accuracy = ac_cv 
    return accuracy

optuna.logging.disable_default_handler()
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

bestparams = study.best_trial.params

train_data = lgb.Dataset(X_train, label=y_train)
eval_data = lgb.Dataset(X_test, label=y_test, reference= train_data)

start = time.time()
model = lgb.train(bestparams,train_data)
print("fit time = {}".format(time.time()-start))

pred = model.predict(X_train)
pred = (pred > 0.5) * 1
ac_train = sklearn.metrics.accuracy_score(y_train, pred)
print(ac_train)

model = lgb.train(bestparams,eval_data)
pred = model.predict(X_test)
pred = (pred > 0.5) * 1
ac_test = sklearn.metrics.accuracy_score(y_test, pred)
print(ac_test)

In [ ]:
sub = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

test = add_label(test)
test["Cabin"].fillna('Z/00/Z',inplace=True)
test["Area"] = test["Cabin"].apply(get_position)
test["Side"] = test["Cabin"].apply(get_position2)
test['GroupId'] = test['PassengerId'].str[:4]
test['PeopleInGroup1'] = test.groupby('GroupId')['PassengerId'].transform('count')
test['PersonalId'] = test['PassengerId'].str[-2:]
test['PeopleInGroup2'] = test.groupby('PersonalId')['PassengerId'].transform('count')
test['isChild'] = test['Age'].apply(lambda x: 1 if x < 10 else 0)
test['isOld'] = test['Age'].apply(lambda x: 1 if x > 60 else 0)

del test["Cabin"], test["Name"]

test.loc[:,list_cat]  = oe.transform(test[list_cat])
test.loc[:,num_col]  = ptr.transform(test[num_col])
num_col = list(set(train.columns)-set(list_cols)-set(list_cat)-set([target]))
test[num_col] = imp_mean.transform(test[num_col])
df_new = pd.DataFrame(ohe.transform(test[list_cols]), 
                      columns=ohe.get_feature_names(), 
                      dtype=np.int64)
test = test.drop(list_cols, axis=1)

df_session_ce_onehot = pd.concat([test, df_new] ,axis=1)

df_session_ce_onehot.head()
X = df_session_ce_onehot[df_session_ce_onehot.columns[df_session_ce_onehot.columns != target]].values
y_pred = model.predict(X,predict_disable_shape_check=True)
y_pred = (y_pred>0.5).astype(bool)
sub[target] = list(y_pred)
sub.to_csv("./submission.csv",index=False)